In [6]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Load and Scale Datasets

In [ ]:
#load dataset
df = pd.read_csv('weather_data.csv')
cols_to_scale = [col for col in df.columns if col not in ['Month', 'Month_Num', 'Location']]

In [23]:
#scale data
df[cols_to_scale] = StandardScaler().fit_transform(df[cols_to_scale])

### PCA

In [ ]:
#run PCA
pca = PCA(n_components=2)
principle_components = pca.fit_transform(df[cols_to_scale])
df["PC1"] = principle_components[:,0]
df["PC2"] = principle_components[:,1]

In [ ]:
#calculate explained variance
explained_variances = pca.explained_variance_ratio_
total_explained = np.sum(explained_variances)

### Neighbor Matching